## Progresso Vacinação COVID-19
---

#### **INTRODUÇÃO**

Os dados contêm as seguintes informações:

**Country** - este é o país para o qual as informações de vacinação são fornecidas;

**Country ISO Code** - código ISO para o país;

**Date** - data para a entrada de dados; para algumas das datas, temos apenas as vacinações diárias, para outras, apenas o total (cumulativo);

**Total number of vaccin ations** - este é o número absoluto de imunizações totais no país;

**Total number of people vaccinated** - uma pessoa, dependendo do esquema de imunização, receberá uma ou mais (normalmente 2) vacinas; em um determinado momento, o número de vacinações pode ser maior do que o número de pessoas;

**Total number of people fully vaccinated** - este é o número de pessoas que receberam todo o conjunto de imunizações de acordo com o esquema de imunização (normalmente 2); em um determinado momento, pode haver um certo número de pessoas que receberam uma vacina e outro número (menor) de pessoas que receberam todas as vacinas no esquema;

**Daily vaccinations (raw)** - para uma determinada entrada de dados, o número de vacinações para aquela data/país;

**Daily vaccinations** - para uma determinada entrada de dados, o número de vacinações para aquela data/país;

**Total vaccinations per hundred** - razão (em porcentagem) entre o número de vacinações e a população total até a data no país;

**Total number of people vaccinated per hundred** - razão (em porcentagem) entre a população imunizada e a população total até a data no país;

**Total number of people fully vaccinated per hundred** - razão (em porcentagem) entre a população totalmente imunizada e a população total até a data no país;

**Number of vaccinations per day** - número de vacinações diárias para aquele dia e país;

**Daily vaccinations per million** - razão (em ppm) entre o número de vacinações e a população total para a data atual no país;

**Vaccines used in the country** - número total de vacinas usadas no país (atualizado);

**Source name** - fonte da informação (autoridade nacional, organização internacional, organização local etc.);

**Source website** - site da fonte da informação;


### Última Atualização
---

In [ ]:
import datetime
import os
import time
with os.scandir("/kaggle/input/covid-world-vaccination-progress") as dir_entries:
    for entry in dir_entries:
        unix_timestamp  = int(entry.stat().st_mtime)
        utc_time = time.gmtime(unix_timestamp)
        print(f"Dataset last time updated: {utc_time.tm_year}-{utc_time.tm_mon}-{utc_time.tm_mday}")
        break
        
ldt = datetime.datetime.now()
print(f"Noteboook last time updated: {ldt.year}-{ldt.month}-{ldt.day}")

### Preparando Análise
---

Inicializamos os pacotes Python que usaremos para ingestão, preparação e visualização de dados. Usaremos principalmente o Plotly para visualização. Então lemos o arquivo de dados e agregamos os dados em alguns campos (país, iso_code e vacinas - que é o esquema de vacinação usado em um determinado país).

Vamos principalmente olhar para:

* Quais esquemas de vacinação são usados ​​em vários países;
* Número total de vacinações e porcentagem de vacinações;
* Vacinações diárias e vacinações diárias por milhão;
* Total de pessoas vacinadas e porcentagem de pessoas vacinadas;

No conjunto de dados também há informações sobre o número total de pessoas completamente vacinadas (e porcentagem delas).

Visualizamos os valores mais recentes (máximos) e também a variação no tempo dos valores mencionados acima.

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

In [4]:
data_df = pd.read_csv("country_vaccinations.csv")

In [ ]:
country_vaccine = data_df.groupby(["country","iso_code","vaccines"])['total_vaccinations','total_vaccinations_per_hundred','daily_vaccinations','daily_vaccinations_per_million','people_vaccinated','people_vaccinated_per_hundred','people_fully_vaccinated','people_fully_vaccinated_per_hundred'].max().reset_index()

country_vaccine.columns = ["Country","iso_code","Vaccines","Total vaccinations","Percent","Daily vaccinations","Daily vaccinations per million","People vaccinated","People vaccinated per hundred",'People fully vaccinated','People fully vaccinated percent']

### Quais vacina são usada em cada País?
---


In [ ]:
vaccines = country_vaccine.Vaccines.unique()
for v in vaccines:
    countries = country_vaccine.loc[country_vaccine.Vaccines==v, 'Country'].values
    print(f"Vaccines: {v}: \nCountries: {list(countries)}\n")

In [ ]:
fig = px.choropleth(locations=country_vaccine['Country'], 
                    locationmode="country names",
                    color=country_vaccine['Vaccines'],
                    title="Countries using each vaccine (different colors for each vaccine)",
                    height = 800
                   )
fig.update_layout({'legend_orientation':'v'})
fig.update_layout({'legend_title':'Vaccine scheme'})
fig.show()

### Qual esquema de vacinação é mais utilizado?
---


#### Geral
Vamos primeiro olhar para o esquema de vacinação usado em geral (não dividido por países).

In [ ]:
vaccine = data_df.groupby(["vaccines"])['total_vaccinations','total_vaccinations_per_hundred',
                                       'daily_vaccinations','daily_vaccinations_per_million'].max().reset_index()
vaccine.columns = ["Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
                           "Daily vaccinations per million"]
def draw_trace_bar_vaccine(data, feature, title, xlab, ylab,color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
            x = data['Vaccines'],
            y = data[feature],
            marker=dict(color=color),
            text=data['Vaccines']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=45, 
                           zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                           showline=True, linewidth=2, linecolor='black', mirror=True,
                          tickfont=dict(
                            size=10,
                            color='black'),), 
              yaxis = dict(title = ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                          showline=True, linewidth=2, linecolor='black', mirror=True),
              plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
              hovermode = 'closest',
              height = 800
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')

Alguns países estão usando um esquema de vacinação misto (eles estão usando mais de uma vacina).

O mapeamento é o seguinte:

* Moderna, Pfizer/BioNTech - EUA;
* CNBG, Sinovac - China;
* Oxford/AstraZeneca, Pfizer/BioNTech', 'Pfizer/BioNTech - Reino Unido;
* Pfizer/BioNTech - principalmente UE;
* Pfizer/BioNTech, Sinopharm - Emirados Árabes Unidos;
* Sinovac - Turquia;
* Covaxin, Covishield - Índia;